# Import Libraries
- nanohub.remote is required to access nanohub web services
- pandas, plotly and numpy are used to visualize data

In [ ]:
import nanohub.remote as nr
import pandas as pd
import json
from plotly.graph_objects import FigureWidget


# Authentication Data
If the services are going to be accessed outside of nanohub, a proper authentication is required
- to get client id and secret, create a web application (https://nanohub.org/developer/api/applications/new), use "https://127.0.0.1" as Redirect URL
- to get username and password, register on nanohub.org (https://nanohub.org/register/)

In [ ]:
auth_data = {
  'client_id': 'XXXXXXXX',
  'client_secret': 'XXXXXXXX',
  'grant_type': 'password',
  'username': 'XXXXXXXX',
  'password': 'XXXXXXXX'
}

Is posible to reuse current nanoHUB session by reading its information from the resources file

In [ ]:
import os
auth_data = {
    'grant_type' : 'tool',
}
with open(os.environ["SESSIONDIR"]+"/resources") as file:
    lines = [line.split(" ", 1) for line in file.readlines()]
    properties = {line[0].strip(): line[1].strip() for line in lines if len(line)==2}
    auth_data["sessiontoken"] = properties["session_token"]
    auth_data["sessionnum"] = properties["sessionid"]

## Web Services Session

Using the authentication data nanohub-remote creates a Generic session (nr.Tools(auth_data))

In [ ]:
session = nr.Session(auth_data)

To view all available tools on the results database use the tools end point

In [ ]:
req_json = session.requestPost('dbexplorer/dbexplorer/tools?simtool=true')
req_json = req_json.json()
pd.DataFrame([p for p in req_json["results"]])

## Query tool tool inputs and outpus
Given an specific tool is posible to query their schema inputs

In [ ]:
SIM2L = "st4pnjunction"
req_json = session.requestPost('dbexplorer/dbexplorer/tool_detail', data={'tool':SIM2L, 'simtool':True})
req_json = req_json.json()

Print its inputs: 

In [ ]:
pd.DataFrame(req_json['results'][0][SIM2L]["input"]).transpose()

In [ ]:
list_inputs = ["input." + i for i in req_json['results'][0][SIM2L]["input"]]
list_inputs

As well as its outputs:

In [ ]:
pd.DataFrame(req_json['results'][0][SIM2L]["output"]).transpose()

In [ ]:
list_outputs = ["output." + i for i in req_json['results'][0][SIM2L]["output"]]
list_outputs

## Results Queries, 
To request data it is necesary to define filters based on the inputs, and specify the results (outputs) that are going to be returned


 - The following query search for data for the tool 'polymod' that contain a number of monomer types (num_zmatrices) equal to 2, and return the Bond scale factor additionally, all queries return the unique identifier (squid) of that result 	

In [ ]:
search = {
    'tool':SIM2L, 
    'simtool':True, 
    'filters':json.dumps([
        {'field':'squid','operation':'=','value':'st4pnjunction/r9/a0fe446ed8b54a50a24b6ff8151a5b1151520ce3'},
    ]),
    'results':json.dumps([
        'input.p_len',
        'input.n_len',
        'input.i_len',
        'output.Equilibrium Potential',
        'output.Equilibrium Ei'
    ]),    
}
req_json = session.requestPost('dbexplorer/dbexplorer/search', data=search)
req_json = req_json.json()
results = req_json['results']
pd.DataFrame(results)

In [ ]:
Potential = results[0]["output.Equilibrium Potential"]
Intrinsic = results[0]["output.Equilibrium Ei"]
FigureWidget(data =[
    dict(
       type="scatter",
        x = Potential["position"],
        y = Potential["delta"],
        name="CV"
    ),
    dict(
       type="scatter",
        x = Intrinsic["position"],
        y = Intrinsic["function"],
        name="Intrinsic Level"
    ),
])

- The following query request the IV charateristics  and doping from pntoy devices based on multiple filters

In [ ]:
N_LEN = results[0]["input.n_len"]
I_LEN = results[0]["input.i_len"]
VOLTAGE = [0,0.6]
TEMPERATURE = [300,300]
MATERIALP = "Si"
IMPURITY = "false"

search = {
    'tool':SIM2L, 
    'simtool':True, 
    'filters':json.dumps([        
        {'field':'input.n_len','value':N_LEN,'operation':'=='},
        {'field':'input.temperature','value':TEMPERATURE[0],'operation':'>='},
        {'field':'input.temperature','value':TEMPERATURE[1],'operation':'<='},
        {'field':'input.i_len','value':I_LEN,'operation':'=='},
        {'field':'input.materialp','value':MATERIALP,'operation':'=='},
        {'field':'input.impurity','value':IMPURITY,'operation':'=='},
        {'field':'input.voltage','value':0,'operation':'=='},
    ]),
    'results':json.dumps([
        'input.Na', 
        'input.Nd',
        'input.n_len',
        'input.i_len',
        'input.p_len',
        'output.Parameters',
        'output.IV Characteristic',
    ])
}
req_json = session.requestPost('dbexplorer/dbexplorer/search', data=search)
req_json = req_json.json()
pd.DataFrame(req_json['results'])

In [ ]:
layout1 = {
    'title' : "IV Characteristics",
    'xaxis':{
        'title' : 'Voltage',
    },    
    'yaxis':{
        'title' : 'Current [A/cm2]',
        'type' : 'log',
    },
}
traces = []
for res in req_json['results']:
    traces.append({
        'type': 'scatter',
        'name': res['squid'],
        'x':res['output.IV Characteristic']['voltage'],
        'y':res['output.IV Characteristic']['function'],
        'text': "N:" + str(res['input.n_len']) + ", P:" + str(res['input.p_len']) + "<BR>" + "Na:" + "{:.2e}".format(res['input.Na']) + ", Nd:" + "{:.2e}".format(res['input.Nd']) 
    })
fig = FigureWidget(traces, layout1)
fig

- The following query calculates statistics for the previous query. without returning the data

In [ ]:
req_json = session.requestPost('dbexplorer/dbexplorer/stats', data=search)
req_json = req_json.json()
pd.DataFrame(req_json['results']).transpose()